In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        if filename == 'train.csv':
            data = pd.read_csv(os.path.join(dirname, filename))
            print(data.head())
        if filename == 'test.csv':
            test_df = pd.read_csv(os.path.join(dirname, filename))
            print(test_df.head())

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

#6036000
data.info()

In [5]:
#4024000
test_df.info()

**Find NA**

In [9]:
data.isnull().sum()

# check
%matplotlib inline
sns.heatmap(data.isnull(), cbar=False)

**Data Analysis**

In [10]:
display(data.describe())

%matplotlib inline
hist = data.hist(bins=30, sharey=True, figsize=(10,10))

In [11]:
corr = data.corr()
ax = sns.heatmap(corr, vmin=-1, vmax=1, center=0, cmap=sns.diverging_palette(20,200,n=200), square=True)

ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right')

# time_step, u_out : correlation with pressure is high,,

**Feature Engineering**

In [12]:
# one-hot-encoding : R, C, u_out, breath_id

one_hot_R = pd.get_dummies(data['R'])
one_hot_R['r5']=one_hot_R[5]
one_hot_R['r20']=one_hot_R[20]
one_hot_R['r50']=one_hot_R[50]
one_hot_R = pd.concat([one_hot_R['r5'], one_hot_R['r20'], one_hot_R['r50']], axis = 1)
one_hot_R.head()

In [13]:
one_hot_C = pd.get_dummies(data['C'])
one_hot_C['c10']=one_hot_C[10]
one_hot_C['c20']=one_hot_C[20]
one_hot_C['c50']=one_hot_C[50]
one_hot_C = pd.concat([one_hot_C['c10'], one_hot_C['c20'], one_hot_C['c50']], axis = 1)
one_hot_C.head()

In [14]:
#one_hot_br = pd.get_dummies(data['breath_id'])
#one_hot_br.head()

In [15]:
# min-max-scalar : u_in, pressure
from sklearn.preprocessing import MinMaxScaler

numeric_features = ['u_in', 'pressure']
scaler = MinMaxScaler()
transformed_data = scaler.fit_transform(data[numeric_features])

trans_df = pd.DataFrame(transformed_data)
trans_df

In [16]:
trans_df['u_in'] = trans_df[0]
trans_df['pressure'] = trans_df[1]
trans_df = pd.concat([trans_df['u_in'],trans_df['pressure']], axis=1)
trans_df.head()

In [17]:
# id(X) breath_id R C time_step(X) u_in u_out(**corr high**) pressure
final_df = pd.concat([trans_df['pressure'], data['breath_id'], one_hot_R['r5'], one_hot_R['r20'], one_hot_R['r50'], one_hot_C['c10'], one_hot_C['c20'], one_hot_C['c50'], trans_df['u_in'], data['u_out']], axis = 1)
final_df.shape
final_df.columns


In [18]:
final_df.to_csv("./train_eda.csv")

In [ ]:
#final_df = pd.read_csv("./train_eda.csv")

In [19]:
# 예측해야 할 변수인 `Risk`를 제거하여 머신러닝 입력값인 x에 저장합니다.
df_x = final_df.drop(columns=['pressure']).to_numpy()

# 예측해야 할 변수 `pressure`만을 선택하여 numpy 형태로 y에 저장합니다.
df_y = final_df['pressure']
df_y = df_y.to_numpy().ravel() # 1 차원 벡터 형태로 출력하기 위해 ravel 사용

test_size=0.3
random_state=42
## end
x_train, x_valid, y_train, y_valid = train_test_split(df_x, df_y, test_size = 0.3, random_state = random_state)


In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_dim=9),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
    ])

model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(x_train, y_train, epochs=30, verbose=2, batch_size = 512)


In [23]:
all_scores = []

val_mse, val_mae = model.evaluate(x_valid, y_valid, verbose=0)  # 검증 세트로 모델 평가
all_scores.append(val_mae)

# 테스트용 데이터의 예측값을 구합니다.
predictions = model.predict(x_valid)

print("테스트 데이터의 mae 값: ", val_mae)

In [24]:
model.save('./model.h5')

In [25]:
test_df.head()
data = test_df

In [26]:
# 예측해야 할 변수인 `Risk`를 제거하여 머신러닝 입력값인 x에 저장합니다.
test_x = data.drop(columns=['id'])
test_x = test_x.drop(columns=['breath_id'])

In [27]:
# one-hot-encoding : R, C, u_out, breath_id

one_hot_R = pd.get_dummies(data['R'])
one_hot_R['r5']=one_hot_R[5]
one_hot_R['r20']=one_hot_R[20]
one_hot_R['r50']=one_hot_R[50]
one_hot_R = pd.concat([one_hot_R['r5'], one_hot_R['r20'], one_hot_R['r50']], axis = 1)
one_hot_R.head()

In [28]:
one_hot_C = pd.get_dummies(data['C'])
one_hot_C['c10']=one_hot_C[10]
one_hot_C['c20']=one_hot_C[20]
one_hot_C['c50']=one_hot_C[50]
one_hot_C = pd.concat([one_hot_C['c10'], one_hot_C['c20'], one_hot_C['c50']], axis = 1)
one_hot_C.head()

In [35]:
# min-max-scalar : u_in, pressure
from sklearn.preprocessing import MinMaxScaler

numeric_features = ['u_in']
scaler = MinMaxScaler()
transformed_data = scaler.fit_transform(data[numeric_features])

trans_df = pd.DataFrame(transformed_data)

trans_df['u_in']=trans_df[0]
trans_df

In [40]:
# id(X) breath_id R C time_step(X) u_in u_out(**corr high**) pressure
final_test = pd.concat([data['breath_id'], one_hot_R['r5'], one_hot_R['r20'], one_hot_R['r50'], one_hot_C['c10'], one_hot_C['c20'], one_hot_C['c50'], trans_df['u_in'], data['u_out']], axis = 1)
final_test.shape
final_test.columns


In [41]:
final_test.to_csv("./test_eda.csv")

In [ ]:
#final_test = pd.read_csv("./test_eda.csv")

In [43]:
""""
id,pressure
1,20
2,23
3,24
etc.
"""

predict = model.predict(final_test)

In [57]:
pred_df = pd.DataFrame(predict)
pred_df['pressure'] = pred_df[0]
pred_df = pred_df.drop(columns=[0])
pred_df.index.name="id"
pred_df.index=pred_df.index + 1
pred_df

In [58]:
pred_df.to_csv("./submission.csv")